In [16]:
import os
os.chdir("G:/tlslab/BIGIRIMANA Innocent/API")
print("Répertoire courant :", os.getcwd())

Répertoire courant : G:\tlslab\BIGIRIMANA Innocent\API


# database.py

In [17]:
%%writefile database.py

"""Database configuration"""
from sqlalchemy import create_engine
from sqlalchemy.orm import declarative_base, sessionmaker
import pyodbc

# Définir les paramètres de connexion
server = 'localhost\\SQLEXPRESS'  # Nom du serveur correct
database = 'Lallemand_oenologie'

# Établir la connexion
connection_string = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};Trusted_Connection=yes;'
conn = pyodbc.connect(connection_string)

print("Connexion réussie!")

# Configuration de SQLAlchemy
SQLALCHEMY_DATABASE_URL = f"mssql+pyodbc://{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes"

# Créer un moteur de base de données (engine) qui établit la connexion avec notre base SQL Server.
engine = create_engine(SQLALCHEMY_DATABASE_URL)

# Définir SessionLocal, qui permet de créer des sessions pour interagir avec la base de données.
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

# Définir Base, qui servira de classe de base pour nos modèles SQLAlchemy.
Base = declarative_base()

# Optionnel : pour exécuter une vérification de la connexion à la base de données
# (peut être utile pour le débogage ou la configuration initiale).
if __name__ == "__main__":
    try:
        with engine.connect() as conn:
            print("Connexion à la base de données réussie.")
    except Exception as e:
        print(f"Erreur de connexion : {e}")

Overwriting database.py


 # models.py

In [18]:
%%writefile models.py
from sqlalchemy import Column, Integer, String, Float, ForeignKey
from sqlalchemy.orm import relationship
from database import Base

class Fermentation(Base):
    __tablename__ = "fermentation"
    __table_args__ = {'extend_existing': True}  # Ajout de cette ligne
    Code = Column(String(255), primary_key=True)
    Souche = Column(String(255))
    Milieu = Column(String(255))
    volume = Column(String)  # Modifié de Text à Float
    Csg_T = Column(Integer)
    donnee = relationship("Donnee", back_populates="ferme", cascade="all, delete")

class Donnee(Base):
    __tablename__ = "donnee"
    __table_args__ = {'extend_existing': True}  # Ajout de cette ligne
    id = Column(Integer, primary_key=True, index=True, autoincrement=True)
    Temps = Column(Float)
    CO2 = Column(Float)
    V= Column(Float)
    Code = Column(String, ForeignKey("fermentation.Code"))
    ferme = relationship("Fermentation", back_populates="donnee")


Overwriting models.py


# TEST models.py et database.py

In [19]:
from database import SessionLocal
from models import Fermentation, Donnee  # Utiliser le nom correct de la classe

db = SessionLocal()


In [20]:

donnees = db.query(Fermentation).limit(10).all()

if donnees:
    for donne in donnees:
        print(f"CODE : {donne.Code}, Souche : {donne.Souche}, Milieu : {donne.Milieu}, Volume : {donne.volume}, Csg_T : {donne.Csg_T }")
else:
    print("No donnes found.")




CODE : A1_2020-08-05 16:18:00_Persy_MSN250 S270 P20, Souche : Persy, Milieu : MSN250 S270 P20, Volume : Robot, Csg_T : 24
CODE : A1_2021-03-03 16:40:00_1410_MSN180 S240 P3, Souche : 1410, Milieu : MSN180 S240 P3, Volume : Robot, Csg_T : 20
CODE : A1_2022-01-11 16:22:00_OT69 A4_MSN100 S180 P3, Souche : OT69 A4, Milieu : MSN100 S180 P3, Volume : Robot, Csg_T : 20
CODE : A1_2022-02-15 15:26:00_Laktia_Syrah PR20, Souche : Laktia, Milieu : Syrah PR20, Volume : Robot, Csg_T : 24
CODE : A1_2022-04-13 15:59:00_DP5_MSN250 S260 P10 SO2, Souche : DP5, Milieu : MSN250 S260 P10 SO2, Volume : Robot, Csg_T : 24
CODE : A1_2022-11-03 15:09:00_6D1_Chardo Pr20 mod, Souche : 6D1, Milieu : Chardo Pr20 mod, Volume : Robot, Csg_T : 27
CODE : A1_2022-11-15 15:19:00_6D1_Merlot PR20 mod, Souche : 6D1, Milieu : Merlot PR20 mod, Volume : Robot, Csg_T : 27
CODE : A1_2023-02-07 10:50:00_EC1118_Merlot PR19 S250, Souche : EC1118, Milieu : Merlot PR19 S250, Volume : Robot, Csg_T : 23
CODE : A1_2023-02-23 14:52:00_M2_M

In [21]:
donne = db.query(Donnee).limit(10).all()

if donne:
    for donn in donne:
        print(f"ID : {donn.id}, TEMPS : {donn.Temps}, CO2 : {donn.CO2}, V : {donn.V}, Code : {donn.Code }")
else:
    print("No donnes found.")

ID : 1099658, TEMPS : 0.0, CO2 : 0.0, V : 0.0, Code : F10_2021-05-07 12:25:00_18-2007_chardo PR20
ID : 1099659, TEMPS : 0.33, CO2 : 0.02, V : 0.0, Code : F10_2021-05-07 12:25:00_18-2007_chardo PR20
ID : 1099660, TEMPS : 0.67, CO2 : 0.1, V : -0.03, Code : F10_2021-05-07 12:25:00_18-2007_chardo PR20
ID : 1099661, TEMPS : 1.0, CO2 : 0.08, V : 0.186, Code : F10_2021-05-07 12:25:00_18-2007_chardo PR20
ID : 1099662, TEMPS : 1.33, CO2 : 0.02, V : 0.253, Code : F10_2021-05-07 12:25:00_18-2007_chardo PR20
ID : 1099663, TEMPS : 1.67, CO2 : 0.06, V : 0.147, Code : F10_2021-05-07 12:25:00_18-2007_chardo PR20
ID : 1099664, TEMPS : 2.0, CO2 : 0.0, V : 0.097, Code : F10_2021-05-07 12:25:00_18-2007_chardo PR20
ID : 1099665, TEMPS : 2.33, CO2 : 0.04, V : 0.036, Code : F10_2021-05-07 12:25:00_18-2007_chardo PR20
ID : 1099666, TEMPS : 2.67, CO2 : 0.08, V : 0.014, Code : F10_2021-05-07 12:25:00_18-2007_chardo PR20
ID : 1099667, TEMPS : 3.0, CO2 : 0.02, V : 0.006, Code : F10_2021-05-07 12:25:00_18-2007_cha

# query_helper.py

In [22]:
%%writefile query_helper.py

from sqlalchemy.orm import Session
from sqlalchemy.orm import joinedload
from typing import Optional

import models

# --- Fermentation ---
def get_fermentation(db: Session, Code: str):
    """Récupère une fermentation par son Code."""
    return db.query(models.Fermentation).filter(models.Fermentation.Code == Code)

# --- Donnee ---
def get_donnee(db: Session, Code: str):
    """Récupère une donnée par son Code."""
    return db.query(models.Donnee).filter(models.Donnee.Code == Code)
    
def get_fermentation_count(db: Session):
    """Retourne le nombre total d'évaluations."""
    return db.query(models.Fermentation).count()

def get_donnee_count(db: Session):
    """Retourne le nombre total de tags."""
    return db.query(models.Donnee).count()
    
def get_donnee_dis_code_count(db: Session):
    """Retourne le nombre total de valeurs distinctes de Code."""
    return db.query(models.Donnee.Code).distinct().count()


Overwriting query_helper.py


# TEST query_helper.py

In [23]:
from database import SessionLocal
import query_helper as query_helper

# Créer une session
db = SessionLocal()

In [24]:
donnees = query_helper.get_fermentation(db, Code="F10_2021-05-07 12:25:00_18-2007_chardo PR20")

if donnees:
    for donne in donnees:
        print(f"CODE : {donne.Code}, Souche : {donne.Souche}, Milieu : {donne.Milieu}, Volume : {donne.volume}, Csg_T : {donne.Csg_T }")
else:
    print("No donnes found.")
db.close() 

CODE : F10_2021-05-07 12:25:00_18-2007_chardo PR20, Souche : 18-2007, Milieu : chardo PR20, Volume : Salle, Csg_T : 20


In [25]:
donn = query_helper.get_donnee(db, Code="F10_2021-05-07 12:25:00_18-2007_chardo PR20")

if donn:
    for don in donn:
        print(f"ID : {don.id}, TEMPS : {don.Temps}, CO2 : {don.CO2}, V : {don.V}, Code : {don.Code }")
else:
    print("No donnes found.")
db.close() 

ID : 1099658, TEMPS : 0.0, CO2 : 0.0, V : 0.0, Code : F10_2021-05-07 12:25:00_18-2007_chardo PR20
ID : 1099659, TEMPS : 0.33, CO2 : 0.02, V : 0.0, Code : F10_2021-05-07 12:25:00_18-2007_chardo PR20
ID : 1099660, TEMPS : 0.67, CO2 : 0.1, V : -0.03, Code : F10_2021-05-07 12:25:00_18-2007_chardo PR20
ID : 1099661, TEMPS : 1.0, CO2 : 0.08, V : 0.186, Code : F10_2021-05-07 12:25:00_18-2007_chardo PR20
ID : 1099662, TEMPS : 1.33, CO2 : 0.02, V : 0.253, Code : F10_2021-05-07 12:25:00_18-2007_chardo PR20
ID : 1099663, TEMPS : 1.67, CO2 : 0.06, V : 0.147, Code : F10_2021-05-07 12:25:00_18-2007_chardo PR20
ID : 1099664, TEMPS : 2.0, CO2 : 0.0, V : 0.097, Code : F10_2021-05-07 12:25:00_18-2007_chardo PR20
ID : 1099665, TEMPS : 2.33, CO2 : 0.04, V : 0.036, Code : F10_2021-05-07 12:25:00_18-2007_chardo PR20
ID : 1099666, TEMPS : 2.67, CO2 : 0.08, V : 0.014, Code : F10_2021-05-07 12:25:00_18-2007_chardo PR20
ID : 1099667, TEMPS : 3.0, CO2 : 0.02, V : 0.006, Code : F10_2021-05-07 12:25:00_18-2007_cha

In [26]:
donnee = query_helper.get_fermentation_count(db)
print(donnee)
db.close() 

4575


In [27]:
donnee = query_helper.get_donnee_dis_code_count(db)
print(donnee)
db.close() 

4575


In [28]:
donnee = query_helper.get_donnee_count(db)
print(donnee)
db.close() 

3443059
